In [1]:
import os
import re
import glob
import datetime
import warnings

import numpy as np
import pandas as pd
import xarray as xr
from netCDF4 import Dataset

import collections, math
from osgeo import gdal, gdal_array, osr

%matplotlib inline

## 1) Read MODIS data (all bands) from NCI

In [2]:
## Rename the bands to something useful and define tile coordinate function
modis_band_map = {
    'Nadir_Reflectance_Band1': 'red_630_690',
    'Nadir_Reflectance_Band2': 'nir1_780_900',
    'Nadir_Reflectance_Band3': 'blue_450_520',
    'Nadir_Reflectance_Band4': 'green_530_610',
    'Nadir_Reflectance_Band5': 'nir2_1230_1250',
    'Nadir_Reflectance_Band6': 'swir1_1550_1750',
    'Nadir_Reflectance_Band7': 'swir2_2090_2350',
}
def add_tile_coords(tile, dataset):
    scale = 1111950.5196669996

    # regex to match string
    regex = re.compile('h\d+v\d+')
    matches = regex.findall(tile)

    # extract values from string
    extract = re.compile('\d+')
    h, v = extract.findall(matches[0])
    h = int(h)
    v = int(v)

    # calculate start and end values
    x_start = scale * (h - 18)
    x_end = scale * (h - 17)

    y_start = -scale * (v - 9)
    y_end = -scale * (v - 8)

    dataset['x'] = xr.IndexVariable('x', np.linspace(x_start, x_end, 2400))
    dataset['y'] = xr.IndexVariable('y', np.linspace(y_start, y_end, 2400))
    return dataset


In [5]:
reflectance_file_cache = sorted(glob.glob(
            '/g/data/u39/public/data/modis/lpdaac-tiles-c6/MCD43A4.006/' +
            '2014.??.??/MCD43A4.A2014???.h30v12.006.*.hdf'         
        ))

len(reflectance_file_cache)

92

In [6]:
## define function 'get_reflectance' whose input is tile ID 'h__v__'
## This function searches the cached list for filenames which contain tile 'h__v__
## outputs data as a concatenated dataset with dimensions, time, x ,y 
def get_reflectance(tile):
    assert re.match(r'h\d\dv\d\d', tile), 'tile must be string "h__v__"'
    files = [f for f in reflectance_file_cache if tile in os.path.basename(f)]
    pattern = re.compile(r'MCD43A4.A(?P<year>\d{4})(?P<day>\d{3}).h\d\dv\d\d.006.\d+.hdf')
    dates, parts = [], []
    for f in files:
        assert os.path.isfile(f)
        parts.append(xr.open_dataset(f, chunks=2400,autoclose=True))
        year, day = pattern.match(os.path.basename(f)).groups()
        dates.append(datetime.date(int(year), 1, 1) +
                     datetime.timedelta(days=int(day) - 1))

    dates = pd.to_datetime(dates)
    dates.name = 'time'

    ds = xr.concat(parts, dates)
    out = xr.Dataset()
    for i in map(str, range(1, 8)):
        key = 'Nadir_Reflectance_Band' + i
        data_ok = ds['BRDF_Albedo_Band_Mandatory_Quality_Band' + i] == 0
        out[modis_band_map[key]] = ds[key].where(data_ok).astype('f4')

    out.rename({'YDim:MOD_Grid_BRDF': 'y',
                'XDim:MOD_Grid_BRDF': 'x'}, inplace=True)
    out.time.encoding.update(dict(
        units='days since 1900-01-01', calendar='gregorian', dtype='i4'))
    for var in out.data_vars:
        out[var].encoding.update(dict(
            shuffle=True, zlib=True, chunks=dict(x=240, y=240, time=6), dtype='f4',
        ))
    return add_tile_coords(tile, out)

In [7]:
%%time
ds14 = get_reflectance('h30v12')

CPU times: user 2.8 s, sys: 2.02 s, total: 4.82 s
Wall time: 16.6 s


/g/data/oe9/software/anaconda/envs/geo/lib/python3.6/site-packages/xarray/core/dtypes.py:23: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if np.issubdtype(dtype, float):
/g/data/oe9/software/anaconda/envs/geo/lib/python3.6/site-packages/xarray/core/dtypes.py:23: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if np.issubdtype(dtype, float):
/g/data/oe9/software/anaconda/envs/geo/lib/python3.6/site-packages/xarray/core/dtypes.py:23: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if np.issubdtype(dtype, float):
/g/data/oe9/software/anaconda/envs/geo/lib/python3.6/site-packages/xarray/core/dty

In [8]:
ds14

<xarray.Dataset>
Dimensions:          (time: 92, x: 2400, y: 2400)
Coordinates:
  * time             (time) datetime64[ns] 2014-01-01 2014-01-05 2014-01-09 ...
  * x                (x) float64 1.334e+07 1.334e+07 1.334e+07 1.334e+07 ...
  * y                (y) float64 -3.336e+06 -3.336e+06 -3.337e+06 -3.337e+06 ...
Data variables:
    red_630_690      (time, y, x) float32 dask.array<shape=(92, 2400, 2400), chunksize=(1, 2400, 2400)>
    nir1_780_900     (time, y, x) float32 dask.array<shape=(92, 2400, 2400), chunksize=(1, 2400, 2400)>
    blue_450_520     (time, y, x) float32 dask.array<shape=(92, 2400, 2400), chunksize=(1, 2400, 2400)>
    green_530_610    (time, y, x) float32 dask.array<shape=(92, 2400, 2400), chunksize=(1, 2400, 2400)>
    nir2_1230_1250   (time, y, x) float32 dask.array<shape=(92, 2400, 2400), chunksize=(1, 2400, 2400)>
    swir1_1550_1750  (time, y, x) float32 dask.array<shape=(92, 2400, 2400), chunksize=(1, 2400, 2400)>
    swir2_2090_2350  (time, y, x) float32 

## 2) Write MODIS data to NetCDF (all bands)

In [9]:
%%time
for year in range(2014, 2015):
    fname = '/g/data/oe9/project/team-drip/h30v12_reflectance_{}.nc'.format(year)
    if os.path.isfile(fname):
        print('already done', fname)
        continue
    try:
        ds14.sel(time=str(year)).to_netcdf(fname)
    except Exception as e:
        print('Year {} failed with {}'.format(year, type(e)))
        print(e)

CPU times: user 1h 44min 58s, sys: 3min 51s, total: 1h 48min 49s
Wall time: 1h 48min 35s


## 3) Open MODIS NetCDFs, create index

In [10]:
ds_all=xr.open_dataset(
    '/g/data/oe9/project/team-drip/modis_h30v12_reflectance/h30v12_reflectance_2014.nc',
    chunks=dict(time=12, x=1200), #chunks should be in multiples of saved chunk size
)
ds_all

<xarray.Dataset>
Dimensions:          (time: 92, x: 2400, y: 2400)
Coordinates:
  * time             (time) datetime64[ns] 2014-01-01 2014-01-05 2014-01-09 ...
  * x                (x) float64 1.334e+07 1.334e+07 1.334e+07 1.334e+07 ...
  * y                (y) float64 -3.336e+06 -3.336e+06 -3.337e+06 -3.337e+06 ...
Data variables:
    red_630_690      (time, y, x) float32 dask.array<shape=(92, 2400, 2400), chunksize=(12, 2400, 1200)>
    nir1_780_900     (time, y, x) float32 dask.array<shape=(92, 2400, 2400), chunksize=(12, 2400, 1200)>
    blue_450_520     (time, y, x) float32 dask.array<shape=(92, 2400, 2400), chunksize=(12, 2400, 1200)>
    green_530_610    (time, y, x) float32 dask.array<shape=(92, 2400, 2400), chunksize=(12, 2400, 1200)>
    nir2_1230_1250   (time, y, x) float32 dask.array<shape=(92, 2400, 2400), chunksize=(12, 2400, 1200)>
    swir1_1550_1750  (time, y, x) float32 dask.array<shape=(92, 2400, 2400), chunksize=(12, 2400, 1200)>
    swir2_2090_2350  (time, y, x) fl

In [11]:
# rename vars for simple calcs
blue = ds_all.blue_450_520
green = ds_all.green_530_610
red = ds_all.red_630_690
nir1 = ds_all.nir1_780_900
nir2 = ds_all.nir2_1230_1250
ndvi = (nir1 - red) / (nir1 + red)
ndvi


<xarray.DataArray (time: 92, y: 2400, x: 2400)>
dask.array<shape=(92, 2400, 2400), dtype=float32, chunksize=(12, 2400, 1200)>
Coordinates:
  * time     (time) datetime64[ns] 2014-01-01 2014-01-05 2014-01-09 ...
  * x        (x) float64 1.334e+07 1.334e+07 1.334e+07 1.334e+07 1.335e+07 ...
  * y        (y) float64 -3.336e+06 -3.336e+06 -3.337e+06 -3.337e+06 ...

## Reproject index for 2014 and Generate NetCDF

In [12]:
#function for reprojecting modis sinusoidal to lat/lon
AffineGeoTransform = collections.namedtuple(
    'GeoTransform', ['origin_x', 'pixel_width', 'x_rot',
                     'origin_y', 'y_rot', 'pixel_height'])


def get_geot(ds):
    """Take an Xarray object with x and y coords; return geotransform."""
    return AffineGeoTransform(*map(float, (
        # Affine matrix - start/step/rotation, start/rotation/step - in 1D
        ds.x[0], (ds.x[-1] - ds.x[0]) / ds.x.size, 0,
        ds.y[0], 0, (ds.y[-1] - ds.y[0]) / ds.y.size
    )))


## set bounding box around data (can be larger than data area
class aus:
    start_lat = -30
    stop_lat = -37
    start_lon = 138
    stop_lon = 153

out_res_degrees = 0.005

ll_geot = AffineGeoTransform(
    origin_x=aus.start_lon, pixel_width=out_res_degrees, x_rot=0,
    origin_y=aus.start_lat, y_rot=0, pixel_height=-out_res_degrees
)

new_shape = (
    math.ceil((aus.start_lat - aus.stop_lat) / out_res_degrees),
    math.ceil((aus.stop_lon - aus.start_lon) / out_res_degrees),
)

ll_coords = dict(
    latitude=np.arange(new_shape[0]) * ll_geot.pixel_height + ll_geot.origin_y,
    longitude=np.arange(new_shape[1]) * ll_geot.pixel_width + ll_geot.origin_x,
    )

# MAGIC - describes the MODIS projection
wkt_str = (
    'PROJCS["Sinusoidal",GEOGCS["GCS_Undefined",DATUM["Undefined",'
    'SPHEROID["User_Defined_Spheroid",6371007.181,0.0]],PRIMEM["Greenwich",0.0],'
    'UNIT["Degree",0.0174532925199433]],PROJECTION["Sinusoidal"],'
    'PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],'
    'PARAMETER["Central_Meridian",0.0],UNIT["Meter",1.0]]"'
)


# Next, define some generically useful functions:


def project_array(array, geot=None):
    """Reproject a tile from Modis Sinusoidal to WGS84 Lat/Lon coordinates.
    Metadata is handled by the calling function.
    """
    # Takes around seven seconds per layer for in-memory Australia mosaics
    if geot is None:
        geot = get_geot(array)
    assert isinstance(geot, AffineGeoTransform)

    def array_to_raster(array, geot):
        ysize, xsize = array.shape  # unintuitive order, but correct!
        dataset = gdal.GetDriverByName('MEM').Create(
            '', xsize, ysize,
            eType=gdal_array.NumericTypeCodeToGDALTypeCode(array.dtype))
        dataset.SetGeoTransform(geot)
        dataset.SetProjection(wkt_str)
        dataset.GetRasterBand(1).WriteArray(array)
        return dataset

    if isinstance(array, xr.DataArray):
        array = array.values
    input_data = array_to_raster(array, geot)

    # Set up the reference systems and transformation
    from_sr = osr.SpatialReference()
    from_sr.ImportFromWkt(wkt_str)
    to_sr = osr.SpatialReference()
    to_sr.SetWellKnownGeogCS("WGS84")

    # Get new geotransform and create destination raster
    dest_arr = np.empty(new_shape)
    dest_arr[:] = np.nan
    dest = array_to_raster(dest_arr, ll_geot)

    # Perform the projection/resampling
    gdal.ReprojectImage(
        input_data, dest,
        wkt_str, to_sr.ExportToWkt(),
        gdal.GRA_NearestNeighbour)
    
    return xr.DataArray(
        dest.GetRasterBand(1).ReadAsArray(),
        dims=('latitude', 'longitude'),
        coords=ll_coords)

In [13]:
for year in range(2014,2015):
    some_ndvi=ndvi.sel(time=str(year))
         
    fname = '/g/data/oe9/project/team-drip/h30v12_ndvi_ll_{}.nc'.format(year)
    if os.path.isfile(fname):
        print('already done', fname)
        continue        
  
    out = xr.concat(
    [project_array(some_ndvi.sel(time=step)) for step in some_ndvi.time], 
    dim=some_ndvi.time)
    
   
    try:
        out.to_netcdf(fname)
    except Exception as e:
        print('Year {} failed with {}'.format(year, type(e)))
        print(e)

/g/data/oe9/software/anaconda/envs/geo/lib/python3.6/site-packages/dask/local.py:271: RuntimeWarning: invalid value encountered in true_divide
  return func(*args2)
